## Reference

* [vit_jax_augreg.ipynb](https://colab.research.google.com/github/google-research/vision_transformer/blob/master/vit_jax_augreg.ipynb)

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
# Load master table from Cloud.
with tf.io.gfile.GFile("gs://vit_models/augreg/index.csv") as f:
    df = pd.read_csv(f)

In [3]:
df.columns

Index(['name', 'ds', 'epochs', 'lr', 'aug', 'wd', 'do', 'sd', 'best_val',
       'final_val', 'final_test', 'adapt_ds', 'adapt_lr', 'adapt_steps',
       'adapt_resolution', 'adapt_final_val', 'adapt_final_test', 'params',
       'infer_samples_per_sec', 'filename', 'adapt_filename'],
      dtype='object')

In [4]:
# How many different pre-training datasets?
df["ds"].value_counts()

i21k       17238
i1k        17136
i21k_30    17135
Name: ds, dtype: int64

Filter based on the following criteria:

* Models should be pre-trained on ImageNet-21k and fine-tuned on ImageNet-1k.
* The final ImageNet-1k validation accuracy should be at least 75%. 
* The transfer resolution should be 224 $\times$ 224.

In [7]:
i21k_i1k_models = df.query("ds=='i21k' & adapt_ds=='imagenet2012'")
models_ge_75 = i21k_i1k_models.query("adapt_final_test >= 0.75 & adapt_resolution==224")
models_ge_75.head()

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
5508,R26+S/32,i21k,300.0,0.001,light0,0.03,0.0,0.0,0.465352,0.465049,...,imagenet2012,0.03,20000,224,0.858102,0.79954,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.0...,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.0...
5509,R26+S/32,i21k,300.0,0.001,light0,0.03,0.0,0.0,0.465352,0.465049,...,imagenet2012,0.01,20000,224,0.855370,0.80182,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.0...,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.0...
5576,R26+S/32,i21k,300.0,0.001,medium2,0.03,0.1,0.1,0.448574,0.448105,...,imagenet2012,0.03,20000,224,0.830393,0.79632,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....
5577,R26+S/32,i21k,300.0,0.001,medium2,0.03,0.1,0.1,0.448574,0.448105,...,imagenet2012,0.01,20000,224,0.821574,0.78796,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....
5644,R26+S/32,i21k,300.0,0.001,light0,0.10,0.0,0.0,0.478398,0.477715,...,imagenet2012,0.03,20000,224,0.852794,0.80074,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.1...,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.1...


In [8]:
models_ge_75["name"].value_counts()

R26+S/32    56
S/16        54
R50+L/32    54
B/16        54
B/32        53
L/16        52
B/8          6
Name: name, dtype: int64

Now, we first fetch the maximum accuracies with respect to a given model type and then we pick the underlying models. 

In [9]:
best_scores_by_model_type = (
    models_ge_75.groupby("name")["adapt_final_test"].max().values
)
results = models_ge_75["adapt_final_test"].apply(
    lambda x: x in best_scores_by_model_type
)
models_ge_75[results].sort_values(by=["adapt_final_test"], ascending=False).head(10)

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
50966,B/8,i21k,300.0,0.001,medium2,0.10,0.0,0.0,0.521426,0.521006,...,imagenet2012,0.01,20000,224,0.891742,0.85948,NaN,NaN,B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_...,B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_...
26011,L/16,i21k,300.0,0.001,medium1,0.10,0.1,0.1,0.512275,0.512275,...,imagenet2012,0.01,20000,224,0.901108,0.85716,304330000.0,228.01,L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...,L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...
24854,B/16,i21k,300.0,0.001,medium2,0.03,0.0,0.0,0.504258,0.503623,...,imagenet2012,0.03,20000,224,0.882454,0.84018,86570000.0,658.56,B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-d...,B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-d...
26861,R50+L/32,i21k,300.0,0.001,medium1,0.10,0.1,0.1,0.514453,0.513877,...,imagenet2012,0.01,20000,224,0.897908,0.83784,110950000.0,1046.83,R50_L_32-i21k-300ep-lr_0.001-aug_medium1-wd_0....,R50_L_32-i21k-300ep-lr_0.001-aug_medium1-wd_0....
6392,R26+S/32,i21k,300.0,0.001,light0,0.03,0.1,0.1,0.477891,0.477471,...,imagenet2012,0.03,20000,224,0.850062,0.80944,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.0...,R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.0...
5849,R26+S/32,i21k,300.0,0.001,medium2,0.10,0.0,0.0,0.462588,0.462373,...,imagenet2012,0.01,20000,224,0.837964,0.80462,36430000.0,1814.25,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....
12410,S/16,i21k,300.0,0.001,light1,0.03,0.0,0.0,0.472676,0.472402,...,imagenet2012,0.03,20000,224,0.841321,0.80462,22050000.0,1508.35,S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do...,S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do...
22610,B/32,i21k,300.0,0.001,medium1,0.03,0.0,0.0,0.473789,0.473525,...,imagenet2012,0.03,20000,224,0.844131,0.79436,88220000.0,3597.19,B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-d...,B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-d...


In [10]:
models_ge_75[results].sort_values(by=["adapt_final_test"], ascending=False).head(10)[
    "adapt_filename"
].values.tolist()

['B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_224',
 'L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224',
 'B_16-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224',
 'R50_L_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224',
 'R26_S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.03-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.03-res_224',
 'R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_224',
 'S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224',
 'B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_224']